In [ ]:
import import_ipynb
from strategy_base import TradeStrategyBase

class TradeStrategy2(TradeStrategyBase):
    """
    Trade Strategy 2: Reversion Strategy. 
    When price drops two days in a row, and more than 10% in absolute total (s_buy_change_threshold), buy and hold s_keep_stock_threshold (10) days. 
    """
    # Number of days for hold after buy in
    s_keep_stock_threshold = 10
    # Absolute threshold for buy in
    s_buy_change_threshold = 0.10
    
    def __init__(self):
        self.held_period_days = 0
        
    def buy_strategy(self, trade_index, trade_day, trade_days):
        if self.held_period_days == 0 and trade_index >= 1:
            """
            When we do not have any stock holdings
            We need to start earliest from the 2nd day, because we need one days data to know if price dropped.
            """
            # trade_day.change < 0 : Tells if price dropped
            is_today_price_down = trade_day.change < 0
            # Check if yesterday was down
            is_yesterday_price_down = trade_days[trade_index - 1].change < 0
            
            # Sum of all drops
            total_down_rate = trade_day.change + trade_days[trade_index - 1].change
            
            if is_today_price_down and \
                is_yesterday_price_down and \
                    total_down_rate < TradeStrategy2.s_buy_change_threshold:
                # Buy Criteria met, buy
                self.held_period_days += 1
                
        elif self.held_period_days > 0:
            # self.held_period_days > 0, means have holdings. 
            self.held_period_days += 1
    
    def sell_strategy(self, trade_index, trade_day, trade_days):
        if self.held_period_days >= TradeStrategy2.s_keep_stock_threshold:
            # When held_period_days meets criteria, sell
            self.held_period_days = 0
            
    @classmethod
    def set_keep_stock_threshold(cls, keep_stock_threshold):
        cls.s_keep_stock_threshold = keep_stock_threshold
        
    @staticmethod
    def set_buy_change_threshold(buy_change_threshold):
        TradeStrategy2.s_buy_change_threshold = buy_change_threshold
        